# Selects the best available device for PyTorch computations

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Data Augmentation

In [ ]:
from libraries.data_augmentation_utils import *
from libraries.keypoints_visualization_utils import *

In [ ]:
bbox_params = A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.0, check_each_transform=True)
keypoint_params = A.KeypointParams(format='xy', label_fields=['keypoint_labels'], remove_invisible=False)

night_filter_low = A.Compose([
    A.ToGray(p=0.7),
    A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.2), contrast_limit=(-0.2, 0.1), p=0.9),
    A.GaussNoise(std_range=(0.039, 0.196), p=0.5),
    A.MotionBlur(blur_limit=(3, 7), p=0.3),
    A.ImageCompression(quality_range=(20,40),p=0.4),
    A.Affine(rotate=(-5, 5), p=0.4),
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_high = A.Compose([
    A.ToGray(p=1.0),
    A.RandomBrightnessContrast(brightness_limit=(-0.5, -0.3), contrast_limit=(-0.3, -0.1), p=1.0),
    A.GaussianBlur(blur_limit=(5, 9), p=0.5),
    A.ISONoise(intensity=(0.2, 0.4), p=0.7),
    A.ImageCompression(quality_range=(5,25), p=0.4),
    A.Affine(rotate=(-10, 10), p=0.4)
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_gamma = A.Compose([
    A.ToGray(p=0.9),
    A.RandomGamma(gamma_limit=(80, 120), p=0.7),
    A.ISONoise(intensity=(0.15, 0.35), p=0.4),
    A.MotionBlur(blur_limit=(3, 7), p=0.3),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, -0.1), contrast_limit=(-0.2, 0.1), p=0.7),
    A.ImageCompression(quality_range=(20,40),p=0.5),
    A.Affine(rotate=(-10, 10), p=0.4)
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_blur = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.0), contrast_limit=(-0.2, 0.05), p=1.0),
    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
    A.ImageCompression(quality_range=(30,50),p=0.4),
    A.Defocus(radius=(3, 5), alias_blur=True, p=0.3),
    A.Affine(rotate=(-5, 5), p=0.3)
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_rotation = A.Compose([
    A.ToGray(p=1.0),
    A.ISONoise(intensity=(0.2, 0.4), p=0.5),
    A.MotionBlur(blur_limit=(5, 11), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.1), contrast_limit=(-0.2, 0.1), p=0.8),
    A.Affine(rotate=(-90, 90), p=1.0)  # YOLO rotation obbligatoria
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_complex = A.Compose([
    A.ToGray(p=1.0),
    A.Downscale(scale_range=(0.3, 0.6), p=0.6),
    A.GaussNoise(std_range=(0.05, 0.2), p=0.5),
    A.ImageCompression(quality_range=(1, 30), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0), contrast_limit=(-0.2, 0.1), p=0.7),
    A.Defocus(radius=(3, 5), alias_blur=True, p=0.3),
    A.Affine(
        translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)},
        rotate=(-5, 5),
        scale=(0.9, 1.1),
        p=0.3
    ),
],keypoint_params=keypoint_params,bbox_params=bbox_params)

night_filter_dropout = A.Compose([
    A.ToGray(p=0.8),
    A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.2), contrast_limit=(-0.3, -0.1), p=1.0),
    A.CoarseDropout(num_holes_range=(1,3),hole_height_range=(30,60),hole_width_range=(30,60),fill=0,p=0.5),
    A.MotionBlur(blur_limit=(3, 7), p=0.3),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=0.2),
    A.Affine(rotate=(-10, 10), p=0.3)
],keypoint_params=keypoint_params,bbox_params=bbox_params)

zoom_out_1 = A.Compose([
    A.RandomScale(scale_limit=(-0.55, -0.55), p=1.0),  # scala fissa: riduzione dell'80%
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=0)
],keypoint_params=keypoint_params,bbox_params=bbox_params)

zoom_out_2 = A.Compose([
    A.RandomScale(scale_limit=(-0.69, -0.68), p=1.0),  # scala fissa: riduzione dell'80%
    A.PadIfNeeded(min_height=512, min_width=512, border_mode=0),
    A.Affine(rotate=(-90, 90), p=1.0)
], keypoint_params=keypoint_params,bbox_params=bbox_params)

night_aug_list = [night_filter_low, night_filter_high, night_filter_gamma, night_filter_blur, night_filter_rotation, night_filter_complex, night_filter_dropout,zoom_out_1, zoom_out_2]

In [ ]:
dt = DataAugmentation(night_aug_list,"/Users/lorenzodimaio/Download/SIDS_revelation_project/data_augmentation/pose700.yolov8",kpt=True,bbox=True)
dt.apply_data_augmentation()

In [ ]:
dt2 = DataAugmentation(night_aug_list,"/Users/lorenzodimaio/Download/SIDS_revelation_project/data_augmentation/face_detection_v3",kpt=False,bbox=True)
dt2.apply_data_augmentation()

In [28]:
draw_keypoints(img_path="/Users/lorenzodimaio/Download/SIDS_revelation_project/data_augmentation/pose700_augm.yolov8/test/images/13_JPG_jpg.rf.24d3294173b86689dc1decf3cb0f10fd.jpg",
               label_kpt_path="/Users/lorenzodimaio/Download/SIDS_revelation_project/data_augmentation/pose700_augm.yolov8/test/labels/13_JPG_jpg.rf.24d3294173b86689dc1decf3cb0f10fd.txt",
               output_path="/Users/lorenzodimaio/Download/SIDS_revelation_project/data_augmentation/immagine_con_keypoints_numerati.jpg",thickness_line=1,thickness_point=3)

